In [1]:
from IPython.display import display, Markdown, clear_output
import utils
import ipywidgets as ipw
import widgets
import json

# Sample preparation

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
DATA_MODEL = utils.read_yaml("/home/jovyan/aiida-openbis/Notebooks/Metadata_Schemas_LinkML/materialMLinfo.yaml")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
SAMPLE_TASKS = ["PREPARATION", "MEASUREMENTS"]

INSTRUMENTS_CONFIG = utils.read_json("instruments_config.json")
INSTRUMENTS_ACTIONS = {}
INSTRUMENTS_OBSERVABLES = {}
for instrument_identifier, instrument_data in INSTRUMENTS_CONFIG.items():
    if instrument_identifier not in INSTRUMENTS_ACTIONS:
        INSTRUMENTS_ACTIONS[instrument_identifier] = {}
    
    if instrument_identifier not in INSTRUMENTS_OBSERVABLES:
        INSTRUMENTS_OBSERVABLES[instrument_identifier] = {}
        
    for component_identifier, component_data in instrument_data["components"].items():
        component_name = component_data["name"]
        component_actions = component_data["actions"]
        component_observables = component_data["observables"]
        
        for action in component_actions:
            if action not in INSTRUMENTS_ACTIONS[instrument_identifier]:
                INSTRUMENTS_ACTIONS[instrument_identifier][action] = [(component_name, component_identifier)]
            else:
                INSTRUMENTS_ACTIONS[instrument_identifier][action].append((component_name, component_identifier))
        
        if component_observables:
            if "Observable" not in INSTRUMENTS_OBSERVABLES[instrument_identifier]:
                INSTRUMENTS_OBSERVABLES[instrument_identifier]["Observable"] = [(component_name, component_identifier)]
            else:
                INSTRUMENTS_OBSERVABLES[instrument_identifier]["Observable"].append((component_name, component_identifier))

In [ ]:
import datetime
import re

class ActionWidget(ipw.VBox):
    def __init__(self, instrument: str = None, history = False, widget_data = None, parent_accordion_widget = None, parent_accordion_item_index = -1):
        super().__init__()
        self.history = history
        self.parent_accordion_widget = parent_accordion_widget
        self.parent_accordion_item_index = parent_accordion_item_index
        self.instrument = instrument
        self.widget_data = widget_data
        instrument_action_obj_classes = list(INSTRUMENTS_ACTIONS[self.instrument].keys())
        instrument_action_types = [("Select action type...", -1)]
        
        for action_obj_class in instrument_action_obj_classes:
            action_obj_label = DATA_MODEL["classes"][action_obj_class]["title"]
            instrument_action_types.append((action_obj_label, action_obj_class))
        
        self.action_dropdown = ipw.Dropdown(
            description = "Action type", 
            options = instrument_action_types, 
            value = instrument_action_types[0][1]
        )
        
        if self.history:
            self.action_dropdown.disabled = True
            self.obj_class = self.widget_data["action_type"]
            self.action_dropdown.value = self.obj_class
            
            self.action_properties_widgets = OpenbisObjectWidget(
                self.obj_class,
                DATA_MODEL,
                disabled = True
            )
            action = utils.get_openbis_object_data(
                OPENBIS_SESSION,
                self.widget_data["identifier"],
                DATA_MODEL
            )
            utils.load_object_widget_values(
                OPENBIS_SESSION, 
                self.action_properties_widgets.properties_widgets,
                action["props"]
            )
            self.components_label = ipw.Label(value = "Components")
            self.components = INSTRUMENTS_ACTIONS[self.instrument][self.obj_class]
            self.components_multiple_selector = utils.SelectMultiple(
                disabled = True,
                options = self.components,
                layout = ipw.Layout(width = '500px', height = '150px'), 
                style = {'description_width': "initial"}
            )
            self.components_settings_label = ipw.Label(value = "Components Settings")
            self.components_settings_accordion = ipw.Accordion()
            
            selected_components = []
            accordion_items = []
            for setting_identifier in self.widget_data["settings"]:
                setting = utils.get_openbis_object_data(OPENBIS_SESSION, setting_identifier, DATA_MODEL)
                component_identifier = setting["props"]["component"]
                selected_components.append(component_identifier)
                
                openbis_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = component_identifier)
                item_title = openbis_object.props["name"]
                self.components_settings_accordion.set_title(len(accordion_items), item_title)
                component_setting_keys = INSTRUMENTS_CONFIG[self.instrument]["components"][component_identifier]["settings"]
                settings_widgets = OpenbisObjectWidget("Setting", DATA_MODEL, properties = component_setting_keys, disabled = True)
                utils.load_object_widget_values(OPENBIS_SESSION, settings_widgets.properties_widgets, setting["props"])
                accordion_items.append(settings_widgets)
                
            self.components_multiple_selector.value = selected_components
            self.components_settings_accordion.children = accordion_items
            
            action_widgets_children = [
                self.action_dropdown,
                self.action_properties_widgets
            ]
            if self.obj_class == "Deposition":
                self.molecule_dropdown = widgets.ObjectSelectionWidget("Molecule", disabled = True)
                self.molecule_dropdown.load_dropdown_box()
                self.molecule_dropdown.dropdown.value = action["props"]["molecule"]
                action_widgets_children.append(self.molecule_dropdown)
            elif self.obj_class == "Dosing":
                self.chemical_dropdown = widgets.ObjectSelectionWidget("Chemical", disabled = True)
                self.chemical_dropdown.load_dropdown_box()
                self.chemical_dropdown.dropdown.value = action["props"]["chemical"]
                action_widgets_children.append(self.chemical_dropdown)
            
            action_widgets_children.append(self.components_label)
            action_widgets_children.append(self.components_multiple_selector)
            action_widgets_children.append(self.components_settings_label)
            action_widgets_children.append(self.components_settings_accordion)
            
            self.children = action_widgets_children
        else:
            self.remove_action_button = utils.Button(
                description = 'Remove action', disabled = False, button_style = 'danger',
                tooltip = 'Remove action', layout = ipw.Layout(width = '150px', height = '25px')
            )
            
            self.action_dropdown.observe(self.load_action_widgets, names = "value")
            self.remove_action_button.on_click(self.remove_action_widget)
            
            if self.widget_data:
                self.action_dropdown.value = self.widget_data["action_type"]
                self.action_properties_widgets.properties_widgets["name"]["value_widget"].value = self.widget_data["name"] 
                self.components_multiple_selector.value = self.widget_data["components"]
                self.components_multiple_selector.disabled = True
            else:
                self.children = [self.action_dropdown, self.remove_action_button]
    
    def load_action_widgets(self, change):
        self.obj_class = self.action_dropdown.value
        self.components = INSTRUMENTS_ACTIONS[self.instrument][self.obj_class]
        self.action_properties_widgets = OpenbisObjectWidget(
            self.obj_class,
            DATA_MODEL
        )
        self.components_label = ipw.Label(value = "Components")
        self.components_multiple_selector = utils.SelectMultiple(
            disabled = False,
            options = self.components,
            layout = ipw.Layout(width = '500px', height = '150px'), 
            style = {'description_width': "initial"}
        )
        self.components_settings_label = ipw.Label(value = "Components Settings")
        self.components_settings_accordion = ipw.Accordion()
        
        action_widgets_children = [
            self.action_dropdown,
            self.action_properties_widgets
        ]
        if self.obj_class == "Deposition":
            self.molecule_dropdown = widgets.ObjectSelectionWidget("Molecule")
            self.molecule_dropdown.load_dropdown_box()
            action_widgets_children.append(self.molecule_dropdown)
        elif self.obj_class == "Dosing":
            self.chemical_dropdown = widgets.ObjectSelectionWidget("Chemical")
            self.chemical_dropdown.load_dropdown_box()
            action_widgets_children.append(self.chemical_dropdown)
        
        action_widgets_children.append(self.components_label)
        action_widgets_children.append(self.components_multiple_selector)
        action_widgets_children.append(self.components_settings_label)
        action_widgets_children.append(self.components_settings_accordion)
        action_widgets_children.append(self.remove_action_button)
        
        self.children = action_widgets_children
        
        # Connect widgets with functions
        self.components_multiple_selector.observe(
            self.add_component_setting_widget, 
            names = "value"
        )
        
        self.action_properties_widgets.properties_widgets["name"]["value_widget"].observe(
            self.update_parent_accordion_title, 
            names = 'value'
        )
        
    def add_component_setting_widget(self, change):
        accordion_items = []
        selected_items = self.components_multiple_selector.value
        selected_items = list(selected_items)
        if selected_items:
            for item_identifier in selected_items:
                openbis_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = item_identifier)
                item_title = openbis_object.props["name"]
                self.components_settings_accordion.set_title(len(accordion_items), item_title)
                component_setting_keys = INSTRUMENTS_CONFIG[self.instrument]["components"][item_identifier]["settings"]
                settings_widgets = OpenbisObjectWidget("Setting", DATA_MODEL, properties = component_setting_keys)
                accordion_items.append(settings_widgets)
        
        self.components_settings_accordion.children = accordion_items
    
    def remove_action_widget(self, b):
        actions_accordion_titles = self.parent_accordion_widget._titles
        actions_accordion_children = self.parent_accordion_widget.children
        actions_accordion_children = list(actions_accordion_children)
        actions_accordion_children.pop(self.parent_accordion_item_index)
        del actions_accordion_titles[str(self.parent_accordion_item_index)]
        self.parent_accordion_widget.children = actions_accordion_children
        
        # Update parent accordion indexes
        actions_accordion_titles = list(actions_accordion_titles.values())
        for idx, action_widget in enumerate(actions_accordion_children):
            if action_widget.parent_accordion_item_index > self.parent_accordion_item_index:
                action_widget.parent_accordion_item_index -= 1
            self.parent_accordion_widget.set_title(idx, actions_accordion_titles[idx])
            
        # Remove the last element title
        self.parent_accordion_widget.set_title(len(actions_accordion_children), "")
        
    
    def update_parent_accordion_title(self, change):
        title = self.action_properties_widgets.properties_widgets["name"]["value_widget"].value
        self.parent_accordion_widget.set_title(
            self.parent_accordion_item_index,
            title
        )

class ObservableWidget(ipw.VBox):
    def __init__(self, instrument: str = None, history = False, widget_data = None, parent_accordion_widget = None, parent_accordion_item_index = -1):
        super().__init__()
        self.history = history
        self.parent_accordion_widget = parent_accordion_widget
        self.parent_accordion_item_index = parent_accordion_item_index
        self.instrument = instrument
        self.widget_data = widget_data
        instrument_observable_obj_classes = list(INSTRUMENTS_OBSERVABLES[self.instrument].keys())
        instrument_observable_types = [("Select observable type...", -1)]
        
        for observable_obj_class in instrument_observable_obj_classes:
            observable_obj_label = DATA_MODEL["classes"][observable_obj_class]["title"]
            instrument_observable_types.append((observable_obj_label, observable_obj_class))
        
        self.observable_dropdown = ipw.Dropdown(
            description = "Observable type", 
            options = instrument_observable_types, 
            value = instrument_observable_types[0][1]
        )
        
        if self.history:
            self.observable_dropdown.disabled = True
            self.obj_class = self.widget_data["observable_type"]
            self.observable_dropdown.value = self.obj_class
            
            self.observable_properties_widgets = OpenbisObjectWidget(
                self.obj_class,
                DATA_MODEL,
                disabled = True
            )
            observable = utils.get_openbis_object_data(
                OPENBIS_SESSION,
                self.widget_data["identifier"],
                DATA_MODEL
            )
            utils.load_object_widget_values(
                OPENBIS_SESSION, 
                self.observable_properties_widgets.properties_widgets,
                observable["props"]
            )
            self.components_label = ipw.Label(value = "Component")
            components = INSTRUMENTS_OBSERVABLES[self.instrument][self.obj_class].copy()
            components.insert(0, ("Select component...", -1))
            self.component_dropdown = widgets.ObjectSelectionWidget("Component", disabled = True)
            self.component_dropdown.load_dropdown_box()
            self.component_dropdown.dropdown.options = components
            
            self.components_settings_label = ipw.Label(value = "Components Settings")
            self.component_settings_accordion = ipw.Accordion()
            
            setting_identifier = self.widget_data["setting"]
            setting = utils.get_openbis_object_data(OPENBIS_SESSION, setting_identifier, DATA_MODEL)
            component_identifier = setting["props"]["component"]
            openbis_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = component_identifier)
            item_title = openbis_object.props["name"]
            self.component_settings_accordion.set_title(0, item_title)
            component_setting_keys = INSTRUMENTS_CONFIG[self.instrument]["components"][component_identifier]["settings"]
            setting_widgets = OpenbisObjectWidget("Setting", DATA_MODEL, properties = component_setting_keys, disabled = True)
            utils.load_object_widget_values(OPENBIS_SESSION, setting_widgets.properties_widgets, setting["props"])
            
            accordion_items = [setting_widgets]
            self.component_dropdown.dropdown.value = component_identifier
            self.component_settings_accordion.children = accordion_items
            
            self.children = [
                self.observable_dropdown,
                self.observable_properties_widgets,
                self.component_dropdown,
                self.component_settings_accordion
            ]
        else:
            self.remove_observable_button = utils.Button(
                description = 'Remove observable', disabled = False, button_style = 'danger',
                tooltip = 'Remove observable', layout = ipw.Layout(width = '150px', height = '25px')
            )
            
            self.observable_dropdown.observe(self.load_observable_widgets, names = "value")
            self.remove_observable_button.on_click(self.remove_observable_widget)
            
            if self.widget_data:
                self.observable_dropdown.value = self.widget_data["observable_type"]
                self.observable_properties_widgets.properties_widgets["name"]["value_widget"].value = self.widget_data["name"] 
                self.component_dropdown.dropdown.value = self.widget_data["component"]
                self.component_dropdown.dropdown.disabled = True
            else:
                self.children = [self.observable_dropdown, self.remove_observable_button]

    def load_observable_widgets(self, change):
        self.obj_class = self.observable_dropdown.value
        self.observable_properties_widgets = OpenbisObjectWidget(
            self.obj_class,
            DATA_MODEL
        )
        components = INSTRUMENTS_OBSERVABLES[self.instrument][self.obj_class].copy()
        components.insert(0, ("Select component...", -1))
        
        self.component_dropdown = widgets.ObjectSelectionWidget("Component")
        self.component_dropdown.load_dropdown_box()
        self.component_dropdown.dropdown.options = components
        self.component_setting_label = ipw.Label(value = "Component Setting")
        self.component_setting_accordion = ipw.Accordion()
        
        self.children = [
            self.observable_properties_widgets,
            self.component_dropdown,
            self.component_setting_label,
            self.component_setting_accordion,
            self.remove_observable_button
        ]
        
        # Connect widgets with functions
        self.component_dropdown.dropdown.observe(
            self.add_component_setting_widget, 
            names = "value"
        )
        
        self.observable_properties_widgets.properties_widgets["name"]["value_widget"].observe(
            self.update_parent_accordion_title, 
            names = 'value'
        )
    
    def add_component_setting_widget(self, change):
        item_identifier = self.component_dropdown.dropdown.value
        if item_identifier == -1:
            self.component_setting_accordion.children = []
        else:
            openbis_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = item_identifier)
            item_title = openbis_object.props["name"]
            self.component_setting_accordion.set_title(0, item_title)
            component_setting_keys = INSTRUMENTS_CONFIG[self.instrument]["components"][item_identifier]["settings"]
            setting_widgets = OpenbisObjectWidget("Setting", DATA_MODEL, properties = component_setting_keys)
            self.component_setting_accordion.children = [setting_widgets]
    
    def remove_observable_widget(self, b):
        observables_accordion_titles = self.parent_accordion_widget._titles
        observables_accordion_children = self.parent_accordion_widget.children
        observables_accordion_children = list(observables_accordion_children)
        observables_accordion_children.pop(self.parent_accordion_item_index)
        del observables_accordion_titles[str(self.parent_accordion_item_index)]
        self.parent_accordion_widget.children = observables_accordion_children
        
        # Update parent accordion indexes
        observables_accordion_titles = list(observables_accordion_titles.values())
        for idx, observable_widget in enumerate(observables_accordion_children):
            if observable_widget.parent_accordion_item_index > self.parent_accordion_item_index:
                observable_widget.parent_accordion_item_index -= 1
            self.parent_accordion_widget.set_title(idx, observables_accordion_titles[idx])
            
        # Remove the last element title
        self.parent_accordion_widget.set_title(len(observables_accordion_children), "")
    
    def update_parent_accordion_title(self, change):
        title = self.observable_properties_widgets.properties_widgets["name"]["value_widget"].value
        self.parent_accordion_widget.set_title(
            self.parent_accordion_item_index,
            title
        )
    
class ProcessStepWidget(ipw.VBox):
    def __init__(self, history = True, widget_data = None, parent_accordion_widget = None, parent_accordion_item_index = -1):
        super().__init__()
        self.history = history
        self.widget_data = widget_data
        self.parent_accordion_widget = parent_accordion_widget
        self.parent_accordion_item_index = parent_accordion_item_index
        
        self.process_step_properties_widgets = OpenbisObjectWidget(
            "ProcessStep",
            DATA_MODEL
        )
        
        self.actions_label = ipw.HTML("<b><span style='font-size:14px;'>Actions</span></b>")
        self.actions_accordion = ipw.Accordion()
        self.observables_label = ipw.HTML("<b><span style='font-size:14px;'>Observables</span></b>")
        self.observables_accordion = ipw.Accordion()
        
        if self.history:
            self.instrument_dropdown = widgets.ObjectSelectionWidget(
                "Instrument",
                disabled = True
            )
            self.instrument_dropdown.load_dropdown_box()
            self.instrument_dropdown.dropdown.value = self.widget_data["instrument"]
            
            # Can be replaced by getting data from a JSON file
            process_step = utils.get_openbis_object_data(
                OPENBIS_SESSION,
                self.widget_data["identifier"],
                DATA_MODEL
            )
            utils.load_object_widget_values(
                OPENBIS_SESSION, 
                self.process_step_properties_widgets.properties_widgets, 
                process_step["props"]
            )
            
            for action_identifier in widget_data["actions"]:
                action = utils.get_openbis_object_data(
                    OPENBIS_SESSION,
                    action_identifier,
                    DATA_MODEL
                )
                action_data = {
                    "name": action["props"]["name"],
                    "identifier": action_identifier,
                    "settings": action["props"]["settings"],
                    "action_type": action["schema_class"]
                }
                self.add_action_widget(
                    None, 
                    action_data = action_data
                )
            
            for observable_identifier in widget_data["observables"]:
                observable = utils.get_openbis_object_data(
                    OPENBIS_SESSION,
                    observable_identifier,
                    DATA_MODEL
                )
                observable_data = {
                    "name": observable["props"]["name"],
                    "identifier": observable_identifier,
                    "setting": observable["props"]["setting"],
                    "observable_type": observable["schema_class"]
                }
                self.add_observable_widget(
                    None, 
                    observable_data = observable_data
                )
            
            self.children = [
                self.process_step_properties_widgets,
                self.instrument_dropdown,
                self.actions_label,
                self.actions_accordion,
                self.observables_label,
                self.observables_accordion
            ]
        else:
            self.remove_process_step_button = utils.Button(
                description = 'Remove process step', disabled = False, 
                button_style = 'danger', tooltip = 'Remove process step', 
                layout = ipw.Layout(width = '150px', height = '25px')
            )
            
            self.remove_process_step_button.on_click(self.remove_process_step_widget)
            self.process_step_properties_widgets.properties_widgets["name"]["value_widget"].observe(
                self.update_parent_accordion_title, 
                names = 'value'
            )
            
            if self.widget_data:
                self.process_step_code = self.widget_data["process_step_code"]
                self.instrument_dropdown = widgets.ObjectSelectionWidget(
                    "Instrument",
                    disabled = True
                )
                self.instrument_dropdown.load_dropdown_box()
                self.instrument_dropdown.dropdown.value = self.widget_data["instrument"]
                self.process_step_properties_widgets.properties_widgets["name"]["value_widget"].value = self.widget_data["name"]
                
                # Add actions
                for action_data in self.widget_data["actions"]:
                    self.add_action_widget(None, action_data = action_data)
                
                # Add observables
                for observable_data in self.widget_data["observables"]:
                    self.add_observable_widget(None, observable_data = observable_data)
                
                self.children = [
                    self.process_step_properties_widgets,
                    self.instrument_dropdown,
                    self.actions_label,
                    self.actions_accordion,
                    self.observables_label,
                    self.observables_accordion,
                    self.remove_process_step_button
                ]
            else:
                self.process_step_code = ""
                self.instrument_dropdown = widgets.ObjectSelectionWidget(
                    "Instrument"
                )
                self.instrument_dropdown.load_dropdown_box()
                self.add_action_button = utils.Button(
                    description = 'Add action', disabled = False, button_style = 'success',
                    tooltip = 'Add action', layout = ipw.Layout(width = '150px', height = '25px')
                )
                self.add_observable_button = utils.Button(
                    description = 'Add observable', disabled = False, button_style = 'success',
                    tooltip = 'Add observable', layout = ipw.Layout(width = '150px', height = '25px')
                )
                self.children = [
                    self.process_step_properties_widgets,
                    self.instrument_dropdown,
                    self.actions_label,
                    self.actions_accordion,
                    self.add_action_button,
                    self.observables_label,
                    self.observables_accordion,
                    self.add_observable_button,
                    self.remove_process_step_button
                ]
                
                self.add_action_button.on_click(self.add_action_widget)
                self.add_observable_button.on_click(self.add_observable_widget)
    
    def add_action_widget(self, b, action_data = None):
        self.instrument = self.instrument_dropdown.dropdown.value
        if self.instrument == -1:
            display(utils.Javascript(data = f"alert('Select an instrument.')"))
        else:
            self.instrument_dropdown.dropdown.disabled = True
            current_actions_accordion_children = self.actions_accordion.children
            current_actions_accordion_children = list(current_actions_accordion_children)
            num_accordion_elements = len(current_actions_accordion_children)
            
            if self.history:
                self.actions_accordion.set_title(num_accordion_elements, action_data["name"])
            else:
                self.actions_accordion.set_title(num_accordion_elements, "")
                
            action_widget = ActionWidget(
                self.instrument, 
                parent_accordion_widget = self.actions_accordion,
                parent_accordion_item_index = num_accordion_elements,
                widget_data = action_data,
                history = self.history
            )
            current_actions_accordion_children.append(action_widget)
            self.actions_accordion.children = current_actions_accordion_children
    
    def add_observable_widget(self, b, observable_data = None):
        self.instrument = self.instrument_dropdown.dropdown.value
        if self.instrument == -1:
            display(utils.Javascript(data = f"alert('Select an instrument.')"))
        else:
            self.instrument_dropdown.dropdown.disabled = True
            current_observables_accordion_children = self.observables_accordion.children
            current_observables_accordion_children = list(current_observables_accordion_children)
            num_accordion_elements = len(current_observables_accordion_children)
            
            if self.history:
                self.observables_accordion.set_title(num_accordion_elements, observable_data["name"])
            else:
                self.observables_accordion.set_title(num_accordion_elements, "")
                
            observable_widget = ObservableWidget(
                self.instrument, 
                parent_accordion_widget = self.observables_accordion,
                parent_accordion_item_index = num_accordion_elements,
                widget_data = observable_data,
                history = self.history
            )
            current_observables_accordion_children.append(observable_widget)
            self.observables_accordion.children = current_observables_accordion_children

    def remove_process_step_widget(self, b):
        process_steps_accordion_titles = self.parent_accordion_widget._titles
        process_steps_accordion_children = self.parent_accordion_widget.children
        process_steps_accordion_children = list(process_steps_accordion_children)
        process_steps_accordion_children.pop(self.parent_accordion_item_index)
        del process_steps_accordion_titles[str(self.parent_accordion_item_index)]
        self.parent_accordion_widget.children = process_steps_accordion_children
        
        # Update parent accordion indexes
        process_steps_accordion_titles = list(process_steps_accordion_titles.values())
        for idx, process_step_widget in enumerate(process_steps_accordion_children):
            if process_step_widget.parent_accordion_item_index > self.parent_accordion_item_index:
                process_step_widget.parent_accordion_item_index -= 1
            self.parent_accordion_widget.set_title(idx, process_steps_accordion_titles[idx])
            
        # Remove the last element title
        self.parent_accordion_widget.set_title(len(process_steps_accordion_children), "")

    def update_parent_accordion_title(self, change):
        title = self.process_step_properties_widgets.properties_widgets["name"]["value_widget"].value
        self.parent_accordion_widget.set_title(
            self.parent_accordion_item_index,
            title
        )
        
class PreparationWidget(ipw.VBox):
    def __init__(self, history = False):
        super().__init__()
        self.process_steps_accordion = ipw.Accordion()
        self.history = history
        
        if self.history:
            self.children = [
                self.process_steps_accordion
            ]
        else:
            self.add_process_step_button = utils.Button(
                description = 'Add process step', disabled = False, 
                button_style = 'success', tooltip = 'Add process step', 
                layout = ipw.Layout(width = '150px', height = '25px')
            )
            self.add_process_template_button = utils.Button(
                description = 'Add process template', disabled = False, 
                button_style = 'success', tooltip = 'Add process step', 
                layout = ipw.Layout(width = '150px', height = '25px')
            )
            add_process_buttons = ipw.HBox([self.add_process_template_button, self.add_process_step_button])
            
            self.children = [
                self.process_steps_accordion,
                add_process_buttons
            ]
            
            self.add_process_step_button.on_click(self.add_process_step_widget)
            self.add_process_template_button.on_click(self.add_process_template_widget)
    
    def add_process_step_widget(self, b, process_step_data = None):
        current_steps_accordion_children = self.process_steps_accordion.children
        current_steps_accordion_children = list(current_steps_accordion_children)
        num_accordion_elements = len(current_steps_accordion_children)
        
        if self.history:
            process_step_title = process_step_data["name"] + " (" + process_step_data["registration_date"] + ")"
            self.process_steps_accordion.set_title(
                num_accordion_elements, 
                process_step_title
            )
        else:
            self.process_steps_accordion.set_title(
                num_accordion_elements, 
                ""
            )
        
        process_step_widget = ProcessStepWidget(
            parent_accordion_widget = self.process_steps_accordion,
            parent_accordion_item_index = num_accordion_elements,
            widget_data = process_step_data,
            history = self.history
        )
            
        current_steps_accordion_children.append(process_step_widget)
        self.process_steps_accordion.children = current_steps_accordion_children
    
    def add_process_template_widget(self, b):
        self.process_template_dropdown = widgets.ObjectSelectionWidget("Protocol")
        self.process_template_dropdown.load_dropdown_box()
        current_preparation_children = list(self.children)
        current_preparation_children.insert(0, self.process_template_dropdown)
        self.children = current_preparation_children
        
        # Connect function to dropdown widget
        self.process_template_dropdown.dropdown.observe(
            self.load_process_template_widget, 
            names = "value"
        )
    
    def load_process_template_widget(self, change):
        process_template_identifier = self.process_template_dropdown.dropdown.value
        if process_template_identifier == -1:
            pass
        else:
            process_template = utils.get_openbis_object_data(
                OPENBIS_SESSION,
                process_template_identifier,
                DATA_MODEL
            )
            
            process_template_data = json.loads(process_template["props"]["protocol_template"])
            for process_step_data in process_template_data:
                process_step_data["process_step_code"] = process_template["props"]["short_name"]
                process_step_data["instrument"] = process_template["props"]["instrument"]
                
                # Add process step using template data
                self.add_process_step_widget(
                    None, 
                    process_step_data = process_step_data
                )
                
            current_preparation_children = list(self.children)
            current_preparation_children.pop(0)
            self.children = current_preparation_children
    
    def load_history(self, widget_data = None):
        for process_step_identifier, process_step_data in widget_data.items():
            # Can be replaced by getting data from a JSON file
            process_step = utils.get_openbis_object_data(
                OPENBIS_SESSION,
                process_step_identifier,
                DATA_MODEL
            )
            process_step_data["name"] = process_step["props"]["name"]
            process_step_data["registration_date"] = process_step["registration_date"]
            process_step_data["identifier"] = process_step_identifier
            self.add_process_step_widget(
                None, 
                process_step_data = process_step_data
            )
    
    def reset_widget(self):
        self.process_steps_accordion = ipw.Accordion()
        
        if self.history:
            self.children = [
                self.process_steps_accordion
            ]
        else:
            self.add_process_step_button = utils.Button(
                description = 'Add process step', disabled = False, 
                button_style = 'success', tooltip = 'Add process step', 
                layout = ipw.Layout(width = '150px', height = '25px')
            )
            self.add_process_template_button = utils.Button(
                description = 'Add process template', disabled = False, 
                button_style = 'success', tooltip = 'Add process step', 
                layout = ipw.Layout(width = '150px', height = '25px')
            )
            add_process_buttons = ipw.HBox([self.add_process_template_button, self.add_process_step_button])
            
            self.children = [
                self.process_steps_accordion,
                add_process_buttons
            ]
            
            self.add_process_step_button.on_click(self.add_process_step_widget)
            self.add_process_template_button.on_click(self.add_process_template_widget)
            
class OpenbisObjectWidget(ipw.VBox):
    def __init__(self, obj_type, data_model, disabled = False, properties = []):
        super().__init__()
        self.data_model = data_model
        self.disabled = disabled
        obj = self.data_model["classes"][obj_type]
        if properties:
            self.properties = properties
        else:
            self.properties = self.get_object_properties(obj)
        self.properties_widgets = {}
        
        for prop in self.properties:
            if prop in self.data_model["slots"]:
                prop_title = self.data_model["slots"][prop]["title"]
                prop_range = self.data_model["slots"][prop]["range"]
                prop_openbis_type = self.data_model["slots"][prop]["annotations"]["openbis_type"]
                prop_multivalued = self.data_model["slots"][prop]["multivalued"]
                prop_widget, prop_value_widget = self.get_property_widget(prop_title, prop_range, prop_openbis_type, prop_multivalued)
                if prop_widget and prop_value_widget:
                    # openBIS property type correction
                    if prop == "name":
                        prop = "name"
                        
                    self.properties_widgets[prop] = {"widget": prop_widget, "value_widget": prop_value_widget}
        
        prop_widgets = []
        for prop, item in self.properties_widgets.items():
            prop_widgets.append(item["widget"])
            
        self.children = prop_widgets
    
    def get_object_properties(self, obj):
        properties = obj["slots"]
        while "is_a" in obj:
            obj_type = obj["is_a"]
            obj = self.data_model["classes"][obj_type]
            properties = obj["slots"] + properties
            
        return properties
    
    def get_property_widget(self, title, range, openbis_type, multivalued):
        widget = None
        value_widget = None
        
        if multivalued == False:
            if openbis_type == "VARCHAR":
                label_widget = ipw.HTML(value = f"{title}")
                
                text_widget = utils.Text(
                    layout = ipw.Layout(width = "150px"), 
                    placeholder = "",
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, text_widget]) # Widget with both label and input box
                value_widget = text_widget # Widget with the value of the property
            
            elif openbis_type == "MULTILINE_VARCHAR":
                label_widget = ipw.HTML(value = f"{title}")
                    
                textarea_widget = utils.Textarea(
                    layout = ipw.Layout(width = "200px", height = "100px"), 
                    placeholder = "",
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, textarea_widget])
                value_widget = textarea_widget
            
            elif openbis_type == "BOOLEAN":
                label_widget = ipw.HTML(value = f"{title}")
                    
                boolean_widget = utils.Checkbox(
                    layout = ipw.Layout(width = "150px"),
                    value = False,
                    indent = False,
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, boolean_widget])
                value_widget = boolean_widget
            
            elif openbis_type == "DATE":
                label_widget = ipw.HTML(value = f"{title}")
                    
                datepicker_widget = ipw.DatePicker(
                    layout = ipw.Layout(width = "150px"), 
                    value = datetime.date.today(),
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, datepicker_widget])
                value_widget = datepicker_widget
            
            elif openbis_type == "TIMESTAMP":
                label_widget = ipw.HTML(value = f"{title}")
                    
                text_widget = utils.Text(
                    layout = ipw.Layout(width = "150px"), 
                    placeholder = "",
                    value = datetime.datetime.now().strftime("%m/%d/%Y %H:%M:%S"),
                    disabled = self.disabled
                )
                        
                widget = ipw.VBox([label_widget, text_widget])
                value_widget = text_widget
            
            elif openbis_type == "INTEGER":
                label_widget = ipw.HTML(value = f"{title}")
                    
                int_widget = utils.Text(
                    layout = ipw.Layout(width = "150px"),
                    disabled = self.disabled
                )
                
                def validate_input(change):
                    """Allow only valid negative or positive integer input."""
                    new_value = change['new']
                    
                    # Check if the input is a valid integer (negative or positive)
                    if new_value == "-" or new_value.lstrip('-').isdigit():
                        int_widget.value = new_value
                    else:
                        # Remove all invalid characters while keeping only a leading '-'
                        cleaned_value = ''.join(filter(str.isdigit, new_value))
                        if new_value.startswith('-'):
                            cleaned_value = '-' + cleaned_value  # Keep '-' at the start if it was originally there
                        int_widget.value = cleaned_value
                
                int_widget.observe(validate_input, names = 'value')
                
                widget = ipw.VBox([label_widget, int_widget])
                value_widget = int_widget
            
            elif openbis_type == "REAL":
                label_widget = ipw.HTML(value = f"{title}")
                    
                float_widget = utils.Text(
                    layout = ipw.Layout(width = "150px"),
                    disabled = self.disabled
                )

                def validate_float_input(change):
                    """Ensure input contains only a valid float format, including negative and scientific notation."""
                    new_value = change['new']

                    # Allow empty value to support gradual input
                    if new_value == "":
                        return  

                    # Strict valid float pattern (final valid numbers)
                    valid_float_pattern = re.compile(r"^-?\d+(\.\d+)?([eE]-?\d+)?$")

                    # Allow intermediate valid inputs while typing
                    intermediate_pattern = re.compile(r"^-?$|^-?\d+\.?$|^-?\d*\.\d*([eE]-?)?$|^-?\d+([eE]-?)?$")

                    if not valid_float_pattern.fullmatch(new_value) and not intermediate_pattern.fullmatch(new_value):
                        float_widget.value = change["old"]  # Revert to previous valid value
                            
                float_widget.observe(validate_float_input, names = 'value')
                
                widget = ipw.VBox([label_widget, float_widget])
                value_widget = float_widget
            
            elif openbis_type == "CONTROLLEDVOCABULARY":
                label_widget = ipw.HTML(value = f"{title}")
                    
                property_options = []
                property_vocabulary = DATA_MODEL["enums"][range]["permissible_values"]
                for key, item in property_vocabulary.items():
                    property_options.append((item["annotations"]["openbis_label"], key))
                
                dropdown_widget = utils.Dropdown(
                    layout = ipw.Layout(width = "100px"), 
                    options = property_options,
                    value = property_options[0][1],
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, dropdown_widget])
                value_widget = dropdown_widget
            
            elif openbis_type == "JSON":
                prop_widgets = {}
                prop_class = self.data_model["classes"][range]
                if "slots" in prop_class:
                    prop_slots = self.get_object_properties(prop_class)
                    prop_widgets = {}
                    prop_value_widgets = {}
                    for slot in prop_slots:
                        slot_title = self.data_model["slots"][slot]["title"]
                        slot_range = self.data_model["slots"][slot]["range"]
                        slot_openbis_type = self.data_model["slots"][slot]["annotations"]["openbis_type"]
                        slot_multivalued = self.data_model["slots"][slot]["multivalued"]
                        widget, value_widget = self.get_property_widget(slot_title, slot_range, slot_openbis_type, slot_multivalued)
                        if widget and value_widget:
                            # openBIS property type correction
                            if slot == "name":
                                slot = "name"
                            prop_widgets[slot] = widget
                            prop_value_widgets[slot] = value_widget
                    
                    label_widget = ipw.HTML(value = f"{title}")
                    json_widget = ipw.HBox(list(prop_widgets.values()))
                    widget = ipw.VBox([label_widget, json_widget])
                    value_widget = prop_value_widgets
                else:
                    label_widget = ipw.HTML(value = f"{title}")
                        
                    text_widget = utils.Text(
                        layout = ipw.Layout(width = "200px"), 
                        placeholder = "",
                        disabled = self.disabled
                    )
                    widget = ipw.VBox([label_widget, text_widget])
                    value_widget = text_widget

            # elif openbis_type == "OBJECT":
            #     prop_widgets = {}
            #     prop_class = self.data_model["classes"][range]
            #     if "slots" in prop_class:
            #         prop_slots = self.get_object_properties(prop_class)
            #         prop_widgets = {}
            #         prop_value_widgets = {}
            #         for slot in prop_slots:
            #             slot_title = self.data_model["slots"][slot]["title"]
            #             slot_range = self.data_model["slots"][slot]["range"]
            #             slot_openbis_type = self.data_model["slots"][slot]["annotations"]["openbis_type"]
            #             slot_multivalued = self.data_model["slots"][slot]["multivalued"]
            #             widget, value_widget = self.get_property_widget(slot_title, slot_range, slot_openbis_type, slot_multivalued)
            #             if widget and value_widget:
            #                 # openBIS property type correction
            #                 if slot == "name":
            #                     slot = "name"
            #                 prop_widgets[slot] = widget
            #                 prop_value_widgets[slot] = value_widget
                    
            #         label_widget = ipw.HTML(value = f"{title}")
            #         object_widget = ipw.VBox(list(prop_widgets.values()))
            #         widget = ipw.Accordion([object_widget])
            #         widget.set_title(0, f"Edit {title}")
            #         widget = ipw.VBox([label_widget, widget])
            #         value_widget = prop_value_widgets
        
        else:
            if openbis_type == "REAL":
                label_widget = ipw.HTML(value = f"{title}")
                
                text_widget = utils.Text(
                    layout = ipw.Layout(width = "150px"), 
                    placeholder = "",
                    disabled = self.disabled
                )
                widget = ipw.VBox([label_widget, text_widget]) # Widget with both label and input box
                value_widget = text_widget # Widget with the value of the property
                
        return widget, value_widget
            

In [ ]:
experiment_selector = widgets.ExperimentSelectionWidget()
experiment_selector.load_dropdown_box()

sample_selector = widgets.ObjectSelectionWidget("Sample", contains_label = False)
sample_selector.load_dropdown_box()

new_sample_processes = PreparationWidget()
sample_preparation_history = PreparationWidget(history = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def get_sample_processes(sample_identifier):
    objects_database = {}
    sample_processes = {}
    sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_identifier)
    objects_stack = [(sample_object, None)]
    
    while objects_stack:
        current_object, child_object = objects_stack.pop()
        if current_object.identifier in objects_database:
            current_object_data = objects_database[current_object.identifier][1]
        else:
            current_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, current_object.permid, DATA_MODEL)
            objects_database[current_object.identifier] = [current_object, current_object_data]
        
        if current_object_data["type"] == "PROCESS_STEP" and current_object_data["permId"] not in sample_processes:
            process_permid = current_object_data["permId"]
            process_instrument = current_object_data["props"]["instrument"]
            
            process_actions = current_object_data["props"]["actions"]
            if process_actions is None:
                process_actions = []
                
            process_observables = current_object_data["props"]["observables"]
            if process_observables is None:
                process_observables = []
                
            process_children = current_object_data["children"]
            for child_identifier in process_children:
                if child_identifier not in objects_database:
                    child_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = child_identifier)
                    child_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, child_identifier, DATA_MODEL)
                    objects_database[child_identifier] = [child_object, child_object_data]
                else:
                    child_object_data = objects_database[child_identifier][1]
            
            if process_actions or process_observables:   
                sample_processes[process_permid] = {
                    "actions": process_actions, 
                    "observables": process_observables,
                    "instrument": process_instrument
                }
        
        for parent_identifier in current_object.parents:
            if parent_identifier not in objects_database:
                parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
                parent_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, parent_identifier, DATA_MODEL)
                objects_database[parent_identifier] = [parent_object, parent_object_data]
            else:
                parent_object = objects_database[parent_identifier][0]
                
            objects_stack.append((parent_object, current_object))
            
    return sample_processes

def load_sample_metadata(change):
    if sample_selector.dropdown.value == -1:
        sample_preparation_history.reset_widget()
    else:
        import time
        # t1 = time.time()
        sample_processes = get_sample_processes(sample_selector.dropdown.value)
        # print(time.time() - t1)
        # t2 = time.time()
        sample_preparation_history.load_history(sample_processes)
        # print(time.time() - t2)
        sample_preparation_history.selected_index = None
        
        last_sample_process = None
        for process_identifier in sample_processes:
            current_process = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = process_identifier)
            if last_sample_process:
                if last_sample_process.registrationDate < current_process.registrationDate:
                    last_sample_process = current_process
            else:
                last_sample_process = current_process
        
        if last_sample_process:
            last_sample_experiment = last_sample_process.experiment
            # Notify the user in case the experiment changed
            if experiment_selector.dropdown.value != last_sample_experiment.permId and experiment_selector.dropdown.value != -1:
                dropdown_options_dict = {key: val for val, key in experiment_selector.dropdown.options}
                previous_experiment_name = dropdown_options_dict.get(experiment_selector.dropdown.value)
                new_experiment_name = dropdown_options_dict.get(last_sample_experiment.permId)
                display(utils.Javascript(data = f"alert('{f'Experiment was changed from {previous_experiment_name} to {new_experiment_name}!'}')"))
                    
            experiment_selector.dropdown.value = last_sample_experiment.permId

def save_processes(b):
    experiment_identifier = experiment_selector.dropdown.value
    sample_identifier = sample_selector.dropdown.value
    if experiment_identifier == -1:
        display(utils.Javascript(data = f"alert('Select an experiment.')"))
    elif sample_identifier == -1:
        display(utils.Javascript(data = f"alert('Select a sample.')"))
    else:
        # Check if sample is being prepared or if it is starting a new preparation based on the children.
        # If the sample has preparation tasks as parents and no measurements, the sample preparation steps 
        # are added to the existent preparation object. If the sample has no preparation tasks or if the sample has 
        # some measurements as children, a new preparation object is created.
        new_preparation = False
        last_preparation_object = None
        sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_selector.dropdown.value)
        objects_stack = [(sample_object, None)]
    
        while objects_stack:
            current_object, child_object = objects_stack.pop()
            current_object_data = utils.get_openbis_object_data(OPENBIS_SESSION, current_object.permid, DATA_MODEL)
            
            if current_object_data["type"] == "PREPARATION":
                if current_object_data["parents"] is None:
                    if last_preparation_object:
                        if last_preparation_object.registrationDate < current_object.registrationDate:
                            last_preparation_object = current_object
                    else:
                        last_preparation_object = current_object
            
            for parent_identifier in current_object.parents:
                parent_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = parent_identifier)
                objects_stack.append((parent_object, current_object))
        
        if sample_object.children:
            for sample_child in sample_object.children:
                sample_child_object = utils.get_openbis_object(OPENBIS_SESSION, sample_child)
                if sample_child_object.type in ["1D_MEASUREMENT", "2D_MEASUREMENT"]:
                    new_preparation = True
        
        if new_preparation:
            last_preparation_object = None
        
        # Save processes
        preparation_object_name = "PREP_" + sample_object.props["name"]
        if last_preparation_object is None:
            last_preparation_object = utils.create_openbis_object(
                OPENBIS_SESSION,
                type = "PREPARATION", 
                experiment = experiment_selector.dropdown.value,
                props = {"name": preparation_object_name}
            )
        
        experiment_openbis = OPENBIS_SESSION.get_experiment(experiment_identifier)
        project_experiment_openbis = experiment_openbis.project.identifier
        
        # Create actions collection if it does not exist
        action_collection_identifier = utils.create_collection(
            OPENBIS_SESSION,
            code = "ACTION_COLLECTION",
            project = project_experiment_openbis,
            type = "COLLECTION",
            props = {"name": "Actions"}
        )
        
        # Create observables collection if it does not exist
        observable_collection_identifier = utils.create_collection(
            OPENBIS_SESSION,
            code = "OBSERVABLE_COLLECTION",
            project = project_experiment_openbis,
            type = "COLLECTION",
            props = {"name": "Observables"}
        )
        
        preparation_props = []
        for process_step_widgets in new_sample_processes.process_steps_accordion.children:
            process_step_code = ""
            process_step_props = utils.get_object_widget_values(process_step_widgets.process_step_properties_widgets.properties_widgets)
            instrument_identifier = process_step_widgets.instrument_dropdown.dropdown.value
            process_step_props["instrument"] = instrument_identifier
            process_step_props["actions"] = []
            for action_widgets in process_step_widgets.actions_accordion.children:
                action_props = utils.get_object_widget_values(action_widgets.action_properties_widgets.properties_widgets)
                action_props["settings"] = []
                for idx, setting_widgets in enumerate(action_widgets.components_settings_accordion.children):
                    setting_props = utils.get_object_widget_values(setting_widgets.properties_widgets)
                    setting_props["component"] = action_widgets.components_multiple_selector.value[idx]
                    
                    component_openbis = utils.get_openbis_object(
                        OPENBIS_SESSION,
                        sample_ident = setting_props["component"]
                    )
                    setting_props["name"] = "Settings_" + utils.convert_datetime_to_string(utils.get_current_datetime())
                    setting_props["name"] = setting_props["name"] + "_" + component_openbis.props["name"]
                    
                    setting_openbis = utils.create_openbis_object(
                        OPENBIS_SESSION,
                        type = "SETTING",
                        experiment = "/EQUIPMENT/ILOG/SETTING_COLLECTION",
                        props = setting_props
                    )
                    setting_identifier = setting_openbis.permId
                    action_props["settings"].append(setting_identifier)
                
                action_type = action_widgets.action_dropdown.value
                if action_type == "Deposition":
                    action_props["molecule"] = action_widgets.molecule_dropdown.dropdown.value
                elif action_type == "Dosing":
                    action_props["chemical"] = action_widgets.chemical_dropdown.dropdown.value
                
                action_openbis_type = DATA_MODEL["classes"][action_type]["annotations"]["openbis_label"].upper().replace(" ", "_")
                action_openbis = utils.create_openbis_object(
                    OPENBIS_SESSION,
                    type = action_openbis_type,
                    experiment = action_collection_identifier,
                    props = action_props
                )
                action_identifier = action_openbis.permId
                process_step_props["actions"].append(action_identifier)
                if process_step_code == "":
                    process_step_code = action_openbis.code[0:4]
                else:
                    process_step_code = process_step_code + ":" + action_openbis.code[0:4]
            
            if len(process_step_widgets.actions_accordion.children) > 1:
                process_step_code = f"[{process_step_code}]"
            
            process_step_props["observables"] = []
            for observable_widgets in process_step_widgets.observables_accordion.children:
                observable_props = utils.get_object_widget_values(observable_widgets.observable_properties_widgets.properties_widgets)
                setting_widgets = observable_widgets.component_setting_accordion.children[0]
                setting_props = utils.get_object_widget_values(setting_widgets.properties_widgets)
                setting_props["component"] = observable_widgets.component_dropdown.dropdown.value
                component_openbis = utils.get_openbis_object(
                    OPENBIS_SESSION,
                    sample_ident = setting_props["component"]
                )
                setting_props["name"] = "Settings_" + utils.convert_datetime_to_string(utils.get_current_datetime())
                setting_props["name"] = setting_props["name"] + "_" + component_openbis.props["name"]
                
                setting_openbis = utils.create_openbis_object(
                    OPENBIS_SESSION,
                    type = "SETTING",
                    experiment = "/EQUIPMENT/ILOG/SETTING_COLLECTION",
                    props = setting_props
                )
                setting_identifier = setting_openbis.permId
                observable_props["setting"] = setting_identifier
                
                observable_openbis = utils.create_openbis_object(
                    OPENBIS_SESSION,
                    type = "OBSERVABLE",
                    experiment = observable_collection_identifier,
                    props = observable_props
                )
                observable_identifier = observable_openbis.permId
                process_step_props["observables"].append(observable_identifier)
            
            process_step_openbis = utils.create_openbis_object(
                OPENBIS_SESSION,
                type = "PROCESS_STEP",
                experiment = experiment_identifier,
                props = process_step_props,
                parents = [last_preparation_object, sample_object]
            )
            
            sample_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = sample_object.permId)
            new_sample_name = sample_object.props["name"] + ":" + process_step_code
            sample_props = {"name": new_sample_name, "exists": True}
            sample_object.props["exists"] = False
            sample_object.save()
            
            sample_object = utils.create_openbis_object(
                OPENBIS_SESSION,
                type = "SAMPLE",
                experiment = sample_object.experiment,
                props = sample_props,
                parents = [process_step_openbis]
            )
            process_step_identifier = process_step_openbis.permId
            preparation_props.append(process_step_identifier)
        
        last_preparation_object = utils.get_openbis_object(OPENBIS_SESSION, sample_ident = last_preparation_object.permId)
        last_preparation_object.props["name"] = "PREP_" + sample_object.props["name"]
        last_preparation_object.save()
        
        display(utils.Javascript(data = f"alert('Processes saved. Loading new sample history...')"))
        
        # Clear widgets
        new_sample_processes.reset_widget()
        sample_selector.load_dropdown_box()
        sample_selector.dropdown.value = sample_object.permId

def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

In [ ]:
display(Markdown("## Select experiment"))
display(experiment_selector)

display(Markdown("## Select sample"))
sample_selector.dropdown.observe(load_sample_metadata, names = 'value')
display(sample_selector)

display(Markdown("## Sample history"))
display(sample_preparation_history)

display(Markdown("## Register new sample processing steps"))
display(new_sample_processes)
create_button.on_click(save_processes)
quit_button.on_click(close_notebook)
display(save_close_buttons_hbox)
display(increase_buttons_size)